In [ ]:
from awkwardql import data, interpreter, interface

events = data.RecordArray({
        "muons": data.ListArray([0, 3, 3, 5], [3, 3, 5, 9], data.RecordArray({
            "pt": data.PrimitiveArray([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]),
            "charge": data.PrimitiveArray([-1, 1, -1, 1, -1, 1, -1, 1, -1]),
            "iso": data.PrimitiveArray([0, 0, 100, 50, 30, 1, 2, 3, 4])
    })),
        "electrons": data.ListArray([0, 5, 6, 8], [5, 6, 8, 12], data.RecordArray({
            "pt": data.PrimitiveArray([1, 2, 3, 4, 5, 100, 30, 50, 1, 2, 3, 4]),
            "charge": data.PrimitiveArray([1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1]),
            "mass": data.PrimitiveArray([10, 10, 10, 10, 10, 5, 15, 15, 9, 8, 7, 6])
        })),
        "met": data.PrimitiveArray([100, 200, 300, 400])
    })

events.setindex()
event_data = data.instantiate(events)

@interface.query
def third_not_in_z():
    r"""
    def mass(one, two) {
        91.2 + one.pt + two.pt # yes this is just a stand in
    }

    third = {
        leptons = electrons union muons
        Z = electrons as (lep1, lep2) union muons as (lep1, lep2) 
                   where lep1.charge != lep2.charge               
                   min by abs(mass(lep1, lep2) - 91.2)
        leptons where count(leptons) >= 3 except [?Z.lep1, ?Z.lep2] max by pt
    }

    hist ?third.pt by regular(100, 0, 250) named "third_pt" titled "Leading other lepton pT"
    """

output, counter = third_not_in_z(event_data)

print(output)

counter["third_pt"].mpl()